# 测试
## 使用复旦大学api测试

In [6]:
import requests
import os
# 代理设置
# os.environ["HTTP_PROXY"] = "http://127.0.0.1:8890"
# os.environ["HTTPS_PROXY"] = "http://127.0.0.1:8890"

org = "fudan-generative-ai"
HF_TOKEN = os.getenv("HUGGINGFACE_API")  # 替换为你的 token


headers = {"Authorization": f"Bearer {HF_TOKEN}"}
# 替换为你的代理地址和端口

# 定义要拉取的资源类型
endpoints = {
    "models": f"https://huggingface.co/api/models?author={org}",
    "datasets": f"https://huggingface.co/api/datasets?author={org}"
}

context = []

for repo_type, url in endpoints.items():
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        repos = response.json()
        for repo in repos:
            if int(repo.get("likes", 0)) > 50 or int(repo.get("downloads", 0)) > 50:
                context.append({
                    "type": repo_type,                        # 模型/数据集/空间
                    "id": repo.get("id"),
                    "likes": repo.get("likes", 0),
                    "downloads": repo.get("downloads", 0),
                    "lastModified": repo.get("lastModified", None),
                    "private": repo.get("private", False)
                })
    else:
        print(f"❌ Error fetching {repo_type}: {response.status_code}")

context

[{'type': 'models',
  'id': 'OpenGVLab/InternVL3_5-8B',
  'likes': 51,
  'downloads': 7826,
  'lastModified': None,
  'private': False},
 {'type': 'models',
  'id': 'OpenGVLab/InternVL3_5-GPT-OSS-20B-A4B-Preview',
  'likes': 61,
  'downloads': 12332,
  'lastModified': None,
  'private': False},
 {'type': 'models',
  'id': 'OpenGVLab/InternVL3_5-241B-A28B',
  'likes': 117,
  'downloads': 2673,
  'lastModified': None,
  'private': False},
 {'type': 'models',
  'id': 'OpenGVLab/InternVL3_5-30B-A3B',
  'likes': 31,
  'downloads': 23024,
  'lastModified': None,
  'private': False},
 {'type': 'models',
  'id': 'OpenGVLab/InternVL3_5-8B-HF',
  'likes': 4,
  'downloads': 1391,
  'lastModified': None,
  'private': False},
 {'type': 'models',
  'id': 'OpenGVLab/InternVL3-1B-hf',
  'likes': 7,
  'downloads': 97782,
  'lastModified': None,
  'private': False},
 {'type': 'models',
  'id': 'OpenGVLab/InternVL3_5-38B',
  'likes': 24,
  'downloads': 6596,
  'lastModified': None,
  'private': False},
 

In [2]:
from langchain_openai import ChatOpenAI
import os
from langchain_core.output_parsers import StrOutputParser

# ucloud api key
API_KEY = os.getenv("UCLOUD_API_KEY")
API_BASE = "https://api.modelverse.cn/v1"
model = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base=API_BASE,
    model="deepseek-ai/DeepSeek-V3-0324")
output_parser = StrOutputParser()


In [8]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template="""

请根据json格式数据中description的内容分类到这些标签，
如果输入jason格式为空请跳过以下内容，直接输出“此组织在huggingface上无like或下载数大于50的项目”：

1. 模型（如大语言模型，多模态模型等）
2. 数据集（各类database或者dataset）


jason数据为：

{context}

请逐个输出每个项目的分类，并以download下载数量排序（最多下载数量的放在最前面），


输出格式按照：

**项目名**
分类：
like数：
download数
最后更新时间：（None的话写None）
url:

在输出前确认一下格式，以及是否有按要求排序，请一次输出全部项目
""")

chain = prompt | model | output_parser
result = chain.invoke({"context":context})
print(result)

根据提供的JSON数据，我将按照要求对每个项目进行分类，并按下载量从高到低排序。以下是分类结果：

**OpenGVLab/InternVL2_5-26B**  
分类：模型  
like数：31  
download数：193035  
最后更新时间：None  
url: https://huggingface.co/OpenGVLab/InternVL2_5-26B  

**OpenGVLab/InternVL3-14B**  
分类：模型  
like数：73  
download数：933553  
最后更新时间：None  
url: https://huggingface.co/OpenGVLab/InternVL3-14B  

**OpenGVLab/InternVL2-2B**  
分类：模型  
like数：73  
download数：939404  
最后更新时间：None  
url: https://huggingface.co/OpenGVLab/InternVL2-2B  

**OpenGVLab/InternVL3-38B**  
分类：模型  
like数：42  
download数：178735  
最后更新时间：None  
url: https://huggingface.co/OpenGVLab/InternVL3-38B  

**OpenGVLab/InternVL3-1B**  
分类：模型  
like数：72  
download数：85243  
最后更新时间：None  
url: https://huggingface.co/OpenGVLab/InternVL3-1B  

**OpenGVLab/InternVL3-78B**  
分类：模型  
like数：216  
download数：441658  
最后更新时间：None  
url: https://huggingface.co/OpenGVLab/InternVL3-78B  

**OpenGVLab/InternVL3-8B**  
分类：模型  
like数：95  
download数：287542  
最后更新时间：None  
url: https://huggingface.co/OpenGVLab/InternVL3

# 生产

In [4]:
huggingface_orgs = """
fudan-generative-ai
rethinklab
USTC-KnowledgeComputingLab
ECNU-CILab
OpenGVLab
stepfun-ai
MiniMaxAI
sensenova
OpenLoong
agibot-world
FourierIntelligence
TigerResearch
infly
MemTensor
Infinigence
"""

In [5]:
import requests
import os
from tqdm import tqdm
# 代理设置
os.environ["HTTP_PROXY"] = "http://127.0.0.1:8890"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:8890"


HF_TOKEN = os.getenv("HUGGINGFACE_API")  # 替换为你的 token


headers = {"Authorization": f"Bearer {HF_TOKEN}"}

huggingface_orgs = huggingface_orgs.strip().split('\n')

for org in tqdm(huggingface_orgs, total=len(huggingface_orgs)):
    endpoints = {
        "models": f"https://huggingface.co/api/models?author={org}",
        "datasets": f"https://huggingface.co/api/datasets?author={org}"
    }

    context = []

    for repo_type, url in endpoints.items():
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            repos = response.json()
            for repo in repos:
                if int(repo.get("likes", 0)) > 50 or int(repo.get("downloads", 0)) > 50:
                    context.append({
                        "type": repo_type,                        # 模型/数据集/空间
                        "id": repo.get("id"),
                        "likes": repo.get("likes", 0),
                        "downloads": repo.get("downloads", 0),
                        "lastModified": repo.get("lastModified", None),
                        "private": repo.get("private", False)
                    })
        else:
            print(f"❌ Error fetching {repo_type}: {response.status_code}")


    result = chain.invoke({"context":context})
    with open(f"HuggingFace/{org}_results.txt", "w", encoding="utf-8") as file:
        file.write(result)



100%|██████████| 15/15 [06:04<00:00, 24.27s/it]
